In [ ]:
## Predict both sentiment and imp words

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.display.max_colwidth = -1
from IPython.display import display

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import tensorflow as tf
print(tf.test.gpu_device_name())
# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
#config = tf.compat.v1.ConfigProto()
#config.gpu_options.allow_growth = True

from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.initializers import TruncatedNormal
from keras.utils.np_utils import to_categorical  


In [ ]:
import re
from sklearn.model_selection import train_test_split, KFold

import time
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import transformers
print(transformers.__version__)
from transformers import TFRobertaModel, RobertaConfig

import tokenizers
print(tokenizers.__version__)
from tokenizers import ByteLevelBPETokenizer



In [ ]:
def jaccard(str1, str2): 
    # Competition metric
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
def jaccard_txt_only(str1, str2): 
    
    a = set([s for s in str1.lower().split() if re.fullmatch(pattern = r'\W+',string=s) is None]) 
    b = set([s for s in str2.lower().split() if re.fullmatch(pattern = r'\W+',string=s) is None]) 
    c = a.intersection(b)
    #print(a,b,c)
    if (len(a) + len(b) - len(c)) > 0:
        return float(len(c)) / (len(a) + len(b) - len(c))
    else:
        return 0


In [ ]:
num_epochs = 3
num_data_points = 30000
max_len = 96
#validation_split = 0.2
batch_size_ = 32
train_bert = True
n_folds = 5
lr = 3e-5
gaussian_noise = 0.03

In [ ]:
#help(BertTokenizer.from_pretrained)
#tokenizer = BertTokenizer.from_pretrained("/kaggle/input/bert-base-uncased/vocab.txt")
#dir(tokenizer)
#help(ByteLevelBPETokenizer)


In [ ]:
tokenizer = ByteLevelBPETokenizer(vocab_file="/kaggle/input/roberta-base-tf/vocab.json",
                                  merges_file="/kaggle/input/roberta-base-tf/merges.txt",
                                  lowercase=True, 
                                  add_prefix_space=True,)
tokenizer

In [ ]:
train = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/train.csv")
train = train.dropna().reset_index(drop=True)

train["text"] = train.text.str.strip().apply(lambda x: " " + " ".join(x.lower().split()))
train["selected_text"] = train.selected_text.str.strip().apply(lambda x: " ".join(x.lower().split()))

train.head(5).T

In [ ]:
# EDA

train["ratio"] = train.selected_text.str.strip().str.len() / train.text.str.strip().str.len()
train["s_txt_len"] = train.selected_text.str.strip().str.len()
train["txt_len"] = train.text.astype('str').apply(lambda x: len(x.split(" ")))

print("*"*30)
display(train.loc[(train.selected_text.str.len()<4) & (train.ratio < 1)])

print("*"*30)
display(train.groupby("sentiment").agg({"ratio":["count","mean"]}))
display(train[(train.sentiment == "neutral") & (train.ratio <= 1)].sort_values("ratio").head(10))

print("*"*30)
display(train.groupby("txt_len").agg({"ratio":["count","mean"]})[:10])
display(train[(train.txt_len < 3) & (train.ratio < 1)].sort_values("ratio").head(10))

print("*"*30)
print(train.txt_len.describe())

del train["ratio"], train["txt_len"], train["s_txt_len"]

In [ ]:
#train.text[train.text.str.contains(r"https?[:/\w+]")]
train["text_http"] = train.text.str.contains(r"(https?[:/\w\.]+)")
train["s_text_http"] = train.selected_text.str.contains(r"(https?[:/\w\.]+)")
display(train.groupby("sentiment").agg({"text_http":"sum", "s_text_http":"sum"}))
del train["s_text_http"], train["text_http"]

#display(train[train.sentiment != "neutral"].text.str.extractall(r"(https?[:/\w\.]+)"))
#display(train[train.sentiment != "neutral"].selected_text.str.extractall(r"(https?[:/\w\.]+)"))

In [ ]:
def overlap(selected_txt, txt, offsets_, tok_len):
    s = txt.index(selected_txt)
    e = len(txt) -1 - txt[::-1].index(selected_txt[::-1])
    #print(s,e)
    start_token, end_token = 0,0
    for i,o in enumerate(offsets_):
        #if i < tok_len-1:
        if (s>=o[0]) & (s < o[1]):
            start_token = i
        if (e>=o[0]) & (e <= o[1]):
            end_token = i
    return (start_token, end_token)

In [ ]:
#x_.offsets

In [ ]:
# "what interview! leave me alone"[16:31]
# x_.offsets

In [ ]:
# overlap('leave me alone', "what interview! leave me alone", x_.offsets, len(x_.ids))

In [ ]:
#tokenizer.id_to_token(tokenizer.token_to_id("neutral"))
print(tokenizer.id_to_token(tokenizer.encode("neutral").ids[0]))
roberta_sentiment_map = {"neutral":tokenizer.encode("neutral").ids[0],
                        "positive":tokenizer.encode("positive").ids[0],
                        "negative":tokenizer.encode("negative").ids[0],
                        "sentiment":tokenizer.encode("sentiment").ids[0] }
roberta_sentiment_map


In [ ]:
sentiment_map = {"neutral":1,
                "positive":2,
                "negative":0,}

rev_sentiment_map = {v:k for k, v in sentiment_map.items()}
print(rev_sentiment_map)

In [ ]:

def Dataloader(textId, text, selected_text, sentiment, tokenizer, max_length):
    
    #tokenizer.enable_padding(pad_id = 1, max_length=max_len-4)
    
    x_input_ids, x_token_type_ids, x_attention_mask, x_tokens = [], [], [], []
    y_s,y_e=[],[]
    tok_len, offsets = [], []
    y_sent = []
    for i in range(len(text)):
        #print(i)
        #text[i] = " ".join(text[i].split())
        #selected_text[i] = " ".join(selected_text[i].split())
        
        x_ = tokenizer.encode(text[i])

        s_ind, e_ind = overlap(selected_text[i], text[i], x_.offsets, len(x_.ids))
                               #len(np.where(x_.ids != 1)[0]))

        start_vec = [0]*max_length
        end_vec = [0]*max_length
        start_vec[s_ind+4] = 1
        end_vec[e_ind+4] = 1

        x_input_ids.append([0] + [ roberta_sentiment_map["sentiment"]] + [2]+ [2] + x_.ids + [2] + [1]*(max_length-len(x_.ids)-5))
        #x_input_ids.append([0] + [ roberta_sentiment_map[sentiment[i]]] + [2]+ [2] + x_.ids + [2] + [1]*(max_length-len(x_.ids)-5))
        x_token_type_ids.append([0]*(max_length))
        x_attention_mask.append([1]*4 + x_.attention_mask + [1] + [0]*(max_length-len(x_.ids)-5))
        tok_len.append( 5+len(x_.ids))
        offsets.append([(0,0)]+[(0,0)]+[(0,0)]+[(0,0)] + x_.offsets+[(0,0)] + [(0,0)]*(max_length-len(x_.ids)-5))
        x_tokens.append(["<s>"]+["sentiment"]+["</s>"]+["</s>"]+x_.tokens+["</s>"])
        y_s.append((start_vec))
        y_e.append((end_vec))
        y_sent.append((sentiment_map[sentiment[i]]))

    return {"textIds":np.asarray(textId), 
            "input_ids":np.asarray(x_input_ids), 
            "token_type_ids":np.asarray(x_token_type_ids), 
            "attention_mask":np.asarray(x_attention_mask), 
            "start_vector":np.asarray(y_s), 
            "end_vector":np.asarray(y_e),
            "tok_len":np.asarray(tok_len),
            "sentiment":np.asarray(sentiment),
            "text":text.tolist(),
            "selected_text":selected_text.tolist(),
            "offsets":offsets,
            "tokens":x_tokens,
            "y_sent": to_categorical(np.asarray(y_sent),num_classes=3),
           }



In [ ]:
robertaConfig = RobertaConfig.from_pretrained("/kaggle/input/roberta-base-tf/config.json", output_hidden_states=True)
#robertaConfig["bos_token_id"] = 0


In [ ]:

# https://huggingface.co/transformers/model_doc/bert.html#tfbertmodel

def model(Max_seq_length = max_len, train_bert = True):
    
    input_ids = layers.Input((Max_seq_length,), dtype = tf.int32)
    token_type_ids = layers.Input((Max_seq_length,), dtype = tf.int32)
    attention_mask = layers.Input((Max_seq_length,), dtype = tf.int32)
    
    tfbert = TFRobertaModel.from_pretrained("/kaggle/input/roberta-base-tf/tf_model.h5",
                                    config=robertaConfig)
    tfbert.roberta.trainable = train_bert
    
    _, _, o3 = tfbert({"input_ids":input_ids, 
                       #"token_type_ids":token_type_ids, 
                       "attention_mask":attention_mask})
    o1 = layers.Concatenate()([o3[-2], o3[-1]])
    #o1 = o3[0]
    o1 = layers.GaussianNoise(gaussian_noise)(o1)
    #o1 = layers.Dropout(0.1)(o1)
    
    h1 = layers.Dense(1, activation = "linear", kernel_initializer=TruncatedNormal(stddev=0.02))(o1)
    start_logits = tf.squeeze(h1, axis = -1, name = "start_logits")

    h2 = layers.Dense(1, activation = "linear", kernel_initializer=TruncatedNormal(stddev=0.002))(o1)
    end_logits = tf.squeeze(h2, axis = -1, name = "end_logits")
    
    h3 = layers.Dense(1, activation = "linear", kernel_initializer=TruncatedNormal(stddev=0.002))(o1)
    sent_logits = tf.squeeze(h3, axis = -1)
    sent_output = layers.Dense(3, activation = "linear", kernel_initializer=TruncatedNormal(stddev=0.002), name = "sentiment_logits")(sent_logits)

    model = models.Model([input_ids, token_type_ids, attention_mask], [start_logits, end_logits, sent_output])
    #print(model.summary())
    return model

In [ ]:
# #mod = model()
# #del mod
# import numpy as np
# import pandas as pd
# df = pd.DataFrame(np.random.randint(0, 3, (10, 2)), columns=list('ab'))
# print(df)
# df.apply(pd.Series.value_counts).T

In [ ]:
optimizer = Adam(learning_rate=lr)
#optimizer = mixed_precision.LossScaleOptimizer(optimizer, loss_scale='dynamic')

In [ ]:
def pred_fn(model, data, threshold =0.1):

    with tf.device('/gpu:0'):
        pred_start, pred_end, pred_sent = model.predict((data["input_ids"],
                                              data["token_type_ids"],
                                              data["attention_mask"]))
        
    return pred_start, pred_end, pred_sent

def decode_fn_basic(data, pred_start, pred_end, pred_sent, threshold =0.1, num_examples_to_show = 0):
    
    # For sentiment
    setiment_pred_actual = pd.DataFrame({"actual": pd.Series(data["y_sent"].argmax(axis=1)).map(rev_sentiment_map),
                                         "predicted": pd.Series(tf.nn.softmax(pred_sent, axis=1).numpy().argmax(axis=1)).map(rev_sentiment_map)})\
                                        .assign(correct = lambda x: (x["actual"] == x["predicted"]).astype('int'))
    print("\nSentiment actual vs predictions count")
    print(pd.crosstab(setiment_pred_actual.actual, setiment_pred_actual.predicted))
    print("\nSentiment predictions accuracy")
    print(setiment_pred_actual.groupby("actual")["correct"].mean())
    print("\n")
    
    predicted_text = []
    current_text = []
    outputs = []
    for ix in range(len(pred_start)):
        c_tok_len = data["tok_len"][ix]
        c_input_ids = data["input_ids"][ix][4:c_tok_len-1]
        c_offsets = data["offsets"][ix][4:c_tok_len-1]
        c_tokens = data["tokens"][ix][4:c_tok_len-1]
        c_pred_start = pred_start[ix,4:c_tok_len-1]
        c_pred_end = pred_end[ix,4:c_tok_len-1]
        c_pred_start_pos = tf.nn.softmax(c_pred_start).numpy().argmax()
        c_pred_end_pos = tf.nn.softmax(c_pred_end).numpy().argmax()
        if ix < num_examples_to_show:
            print()
            #print(c_input_ids)
            print(tokenizer.decode(c_input_ids))
            print(c_tokens)
            print(c_pred_start)
            print(c_pred_end)
            print(c_pred_start_pos,c_pred_end_pos)
            print(c_tokens[c_pred_start_pos:c_pred_end_pos+1])
            print(c_offsets)
        # Adjust for words that break into separate tokens
        if c_pred_end_pos < c_pred_start_pos:
            s_e_options = []
            s_e_options.append(((c_pred_start_pos,c_pred_start_pos), 
                                c_pred_start[c_pred_start_pos] + c_pred_end[c_pred_start_pos] ))
            s_e_options.append(((c_pred_end_pos,c_pred_end_pos), 
                                c_pred_start[c_pred_end_pos] + c_pred_end[c_pred_end_pos] ))
            s_e_options.append(((0,len(c_pred_end)-1), 
                                c_pred_start[0] + c_pred_end[len(c_pred_end)-1] ))
            if ix < num_examples_to_show:
                print("c_pred_end_pos < c_pred_start_pos  - ", c_pred_start_pos, c_pred_end_pos)
                print(sorted(s_e_options, key = lambda x: x[1], reverse = True))
            
            c_pred_start_pos, c_pred_end_pos = sorted(s_e_options, key = lambda x: x[1], reverse = True)[0][0]
            
            #c_pred_end_pos = c_pred_start_pos
            
        if (data["sentiment"][ix] == "neutral") | ( len(data["text"][ix].split()) < 2 ):
            c_selected_text = data["text"][ix]
        else:
            c_selected_text = data["text"][ix][c_offsets[c_pred_start_pos][0]:c_offsets[c_pred_end_pos][1]]
        if ix < num_examples_to_show:
            print(c_pred_start_pos,c_pred_end_pos)
            print(c_tokens[c_pred_start_pos:c_pred_end_pos+1])
            print(data["sentiment"][ix])
            print(c_selected_text)
        outputs.append(c_selected_text)
    return outputs, setiment_pred_actual.predicted.values


def eval_fn(preds, labels):
    j=[]
    j_txt = []
    for ix in range(len(preds)):
        j.append( jaccard(preds[ix], labels[ix]) )
        j_txt.append( jaccard_txt_only(preds[ix], labels[ix]) )
        
    return np.mean(j), np.mean(j_txt)


**Train Model on 5-fold data**

In [ ]:
# Read Test data

test = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/test.csv")
test["text"] = test.text.str.strip().apply(lambda x: " " + x.lower())
display(test.loc[test.text.str.contains("   ")])
print(len(test))

test_data = Dataloader(test.textID.values, 
                       test.text.values, 
                       test.text.values, 
                       test.sentiment.values, 
                       tokenizer, 
                       max_len)


In [ ]:

test_data["y_sent"]


In [ ]:
# Create folds
kf = KFold(n_splits = n_folds , shuffle = True, random_state=42)
fold_pred = train.copy()
fold_pred["pred_text"] = ""
fold_pred["pred_sentiment"] = ""

test_pred_start = np.zeros((len(test),max_len), dtype=np.float32)
test_pred_end = np.zeros((len(test),max_len), dtype=np.float32)

for fold, (tr_idx, val_idx) in enumerate(kf.split(train[:num_data_points])):
    best_score = 0
    # Split data
    print("\n******************************* For Fold {} ***********************************".format(fold+1))
    train_data = Dataloader(train.iloc[tr_idx].textID.values, 
                               train.iloc[tr_idx].text.values, 
                               train.iloc[tr_idx].selected_text.values, 
                               train.iloc[tr_idx].sentiment.values, 
                               #np.asarray(pd.Series(["sentiment"]*train.iloc[tr_idx].shape[0])),
                               tokenizer, 
                               max_len)

    val_data = Dataloader(train.iloc[val_idx].textID.values, 
                               train.iloc[val_idx].text.values, 
                               train.iloc[val_idx].selected_text.values, 
                               train.iloc[val_idx].sentiment.values, 
                               #np.asarray(pd.Series(["sentiment"]*train.iloc[val_idx].shape[0])),
                               tokenizer, 
                               max_len)

    # Build and compile model
    tweet_model = model(max_len, train_bert)
    tweet_model.compile(optimizer= optimizer, 
                        loss = CategoricalCrossentropy(from_logits=True))
    
    # Sample weights for neutral
    #sample_weights = np.ones(train_data["input_ids"][:,1].shape)
    #sample_weights[train_data["sentiment"] == "neutral"] = 0.5
    #sample_weights
    
    # Model training 

    with tf.device('/gpu:0'):
        for e in range(num_epochs):
            st = time.time()
            tweet_model.fit((train_data["input_ids"], train_data["token_type_ids"], train_data["attention_mask"]),
                            (train_data["start_vector"],train_data["end_vector"],train_data["y_sent"]),
                            epochs=1,
                            batch_size=batch_size_,
                            shuffle=True,
                            #sample_weight = [sample_weights,sample_weights],
                            )
            print("Time taken {} mins".format(str((time.time()-st)/60)[:5]))
            #break
            val_pred_s,val_pred_e, val_pred_sent = pred_fn(tweet_model, val_data, threshold =0.1)
            val_preds, val_sents = decode_fn_basic(val_data, val_pred_s, val_pred_e, val_pred_sent, threshold =0.1)
            current_score, current_score_txt = eval_fn(val_preds, val_data["selected_text"])
            print("Jaccard score: {}".format( current_score ))
            print("Jaccard score (Alphanumeric only): {}".format( current_score_txt ))
            #break

            if current_score > best_score: # For final fold epoch
                best_score = current_score_txt
                tweet_model.save_weights("/kaggle/working/" + f'fold_{fold+1}.h5')
                # Fill validation predictions
                fold_pred.loc[val_idx, "pred_text"] = val_preds
                fold_pred.loc[val_idx, "pred_sentiment"] = val_sents
                # Fill test predictions
                test_pred_s, test_pred_e, test_pred_sent = pred_fn(tweet_model, test_data, threshold =0.1)
            print()
        #break
    test_pred_start = test_pred_start + test_pred_s / n_folds
    test_pred_end = test_pred_end + test_pred_e / n_folds
    #break

print("\nFor full fold pred dataset - Jaccard score is - ")
print(eval_fn(fold_pred["pred_text"].apply(lambda x: x.strip()), fold_pred["selected_text"]))

# Decode test predictions
test_preds, test_sents = decode_fn_basic(test_data, test_pred_start, test_pred_end, test_pred_sent, threshold = 0.1)


In [ ]:
fold_pred.to_csv('fold_pred.csv', index=False)

In [ ]:

for v in range(10):
    #print(key)
    print(val_data["tokens"][v])
    #print(len(val_data["tokens"][v]))
    #print("attention length : {}".format(sum(val_data["attention_mask"][v])))
    print(val_data["text"][v])
    print(val_data["selected_text"][v])
    print(val_data["start_vector"][v].argmax())
    print(val_data["end_vector"][v].argmax())
    print(val_data["tokens"][v][val_data["start_vector"][v].argmax():val_data["end_vector"][v].argmax()+1])
    print(val_data["offsets"][v][4:val_data["end_vector"][v].argmax()+2])
    print(val_data["text"][v][val_data["offsets"][v][val_data["start_vector"][v].argmax()][0]:val_data["offsets"][v][val_data["end_vector"][v].argmax()][1]+1])
    print()

In [ ]:
# # Analysis of bad predictions 
# val_analysis = pd.DataFrame({"text":fold_pred["text"], 
#                              "pred_text":fold_pred["pred_text"], 
#                              "selected_text": fold_pred["selected_text"],
#                              "sentiment" : fold_pred["sentiment"]})
# #print(val_analysis.dtypes)
# val_analysis["jaccard"] = val_analysis.apply(lambda row: jaccard(row["pred_text"], row["selected_text"]), axis=1)

# for sent in ["positive","negative","neutral"]:
#     print("*"*40 + sent + "*"*40)
#     display(val_analysis[val_analysis.sentiment==sent][["selected_text","pred_text","jaccard"]].sort_values("jaccard",ascending =False).head(10))
#     print()
#     display(val_analysis[(val_analysis.sentiment==sent) & (val_analysis.jaccard < 1)][["selected_text","pred_text","jaccard"]].sort_values("jaccard",ascending =False).head(20))
#     print()
#     display(val_analysis[(val_analysis.sentiment==sent) ][["selected_text","pred_text","jaccard"]].sort_values("jaccard").head(10))
#     print()
#     display(val_analysis[(val_analysis.sentiment==sent) & (val_analysis.jaccard > 0.5) ][["selected_text","pred_text","jaccard"]].sort_values("jaccard").head(20))
#     print()
#     display(val_analysis[(val_analysis.sentiment==sent) & (val_analysis.jaccard > 0) ][["selected_text","pred_text","jaccard"]].sort_values("jaccard").head(20))

In [ ]:
sub = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/sample_submission.csv")
sub["selected_text"] = [t.strip() for t in test_preds]
sub["pred_sentiment"] = test_sents


sub.head(20)

In [ ]:
sub.to_csv('test_predictions.csv', index=False)